## Langchain (chain)

* Um "chain" (ou "cadeia") é uma sequência de componentes ou operações encadeadas para realizar uma tarefa complexa de processamento de linguagem natural (NLP) de forma modular e fluida
* É uma sequência de chamadas que podem envolver diferentes módulos do LangChain (como LLMs, ferramentas, memória, etc.), onde a saída de uma etapa pode ser utilizada como entrada para a próxima. Isso permite a construção de fluxos de trabalho mais sofisticados e poderosos, sem a necessidade de implementar tudo manualmente.

In [1]:
# Bibliotecas

from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversation.base import ConversationChain # ConversationChain
from langchain.prompts import PromptTemplate # Com prompt template
from langchain.chains.llm import LLMChain # Com LLMChain
from langchain.chains import SimpleSequentialChain # Com SimpleSequentialChain
from langchain.chains import SequentialChain # SequentialChain

In [2]:
chat = ChatOpenAI(model='gpt-3.5-turbo-0125')
memory = ConversationBufferMemory()
chain = ConversationChain(
    llm=chat,
    memory=memory,
    verbose=True
)

#### Demonstração geral

In [3]:
# Demonstração 1

chain.predict(input='Hello!')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


'Hello! How are you today?'

In [4]:
# Demonstração 2

chain.run('Hello!')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello!
AI: Hello! How are you today?
Human: Hello!
AI:


/home/debora/Downloads/llm-experiments/m2_langchain/.venv1/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.


"Hello again! Did you know that the average temperature on Earth is about 57 degrees Fahrenheit? It's quite comfortable for most living organisms."

#### Prompt template

In [5]:
# Demonstração 3 - Com prompt template

prompt_template = PromptTemplate.from_template("""
Essa é uma conversa amigável entre um humano e uma IA

conversa atual:
{history}

Human: {input}

AI:
""")

In [6]:
chat = ChatOpenAI(model='gpt-3.5-turbo-0125')
memory = ConversationBufferMemory()
chain = ConversationChain(
    prompt=prompt_template,
    llm=chat,
    memory=memory,
    verbose=True
)

In [7]:
chain.predict(input='Oi')



> Entering new ConversationChain chain...
Prompt after formatting:

Essa é uma conversa amigável entre um humano e uma IA

conversa atual:


Human: Oi

AI:


> Finished chain.


'Olá! Como posso te ajudar hoje?'

#### LLMChain

In [8]:
# Demonstração 4 - com LLMChain

chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [9]:
prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    
    """
)

chain = LLMChain(llm=chat, prompt=prompt)
produto = 'LLMs com LangChain'
chain.run(produto)

'LangChain Solutions Ltd.'

#### SimpleSequentialChain

In [10]:
# Demonstração 5 - com SimpleSequentialChain

prompt = PromptTemplate.from_template(
    """
    Escolha o melhor nome para mim sobre uma empresa que desenvolve soluções em {produto}
    
    """
)

chain_nome = LLMChain(llm=chat, prompt=prompt)

In [11]:
prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com nome {nome_empresa},
    informe uma descrição de até 20 palavras.
    """
)

chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao)

In [12]:
chain = SimpleSequentialChain(
    chains=[chain_nome,chain_descricao],
    verbose=True
)

chain.run(produto)



> Entering new SimpleSequentialChain chain...
"LangTech Solutions"
A LangTech Solutions é uma empresa dedicada a fornecer soluções tecnológicas inovadoras e eficientes para diversas áreas de negócio.

> Finished chain.


'A LangTech Solutions é uma empresa dedicada a fornecer soluções tecnológicas inovadoras e eficientes para diversas áreas de negócio.'

#### SequentialChain

In [13]:
# Demonstração 6


prompt = PromptTemplate.from_template(
    """
    Qual um bom nome de empresa que desenvolve soluções em {produto}?
    """
)
chain_nome = LLMChain(llm=chat, prompt=prompt, output_key="nome_empresa")

prompt_descricao = PromptTemplate.from_template(
    """
    Sobre a empresa com o nome {nome_empresa} e que 
    produz seguinte {produto}.
    Informe uma descrição de até 30 palavras de atividades dessa empresa
    """
)
chain_descricao = LLMChain(llm=chat, prompt=prompt_descricao, 
                           output_key="descricao_empresa")

prompt_traducao = PromptTemplate.from_template(
    """
    Crie um nome em espanhol para a empresa de nome {nome_empresa}
    que possui a seguinte descrição {descricao_empresa}
    """
)
chain_traducao = LLMChain(llm=chat, prompt=prompt_traducao, 
                          output_key="nome_espanhol")

In [14]:
chain = SequentialChain(
    chains=[chain_nome, chain_descricao, chain_traducao],
    input_variables=["produto"],
    output_variables=["nome_empresa", "descricao_empresa", "nome_espanhol"],
    verbose=True
)

resposta = chain.invoke({"produto": produto})



> Entering new SequentialChain chain...

> Finished chain.


In [15]:
resposta["produto"]

'LLMs com LangChain'

In [16]:
resposta["nome_empresa"]

'"LangLink Solutions"'

In [17]:
resposta["descricao_empresa"]

'A LangLink Solutions é uma empresa especializada na produção de LLMs com LangChain, oferecendo soluções avançadas de linguagem para diversas aplicações.'

In [18]:
resposta["nome_espanhol"]

'Soluciones de Lenguaje LangLink'